In [1]:
import torch
from torch import nn
from diffusion_policy.dataset.base_dataset import BaseImageDataset, LinearNormalizer
from consistency_policy.utils import get_policy, rmat_to_quat, rot6d_to_rmat
from consistency_policy.policy_wrapper import PolicyWrapper
import numpy as np

/juno/u/aadityap/miniforge3/envs/robodiff-retry/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ckpt_path = '/juno/u/aadityap/Consistency-Accelerated-Policy-Networks/outputs/ctm/jimmy/checkpoints/epoch=0200-val_mse_error=0.228.ckpt'

In [3]:
policy = get_policy(ckpt_path)

You are using the CTM base workspace! Ensure that you don't wish to use the normal DP base workspace.
Using scheduler singular
Using CTM scheduler

============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['arm_quat', 'arm_pos', 'base_pose', 'gripper_pos']
using obs modality: rgb with keys: ['base_image', 'wrist_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/juno/u/aadityap/miniforge3/envs/robodiff-retry/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/juno/u/aadityap/miniforge3/envs/robodiff-retry/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Inference mode or invalid teacher time embed! You should be doing inference only!
Using losses:  {'ctm': 1, 'dsm': 1}
CM params: 2.594075e+08
Teacher params: 2.557375e+08
Vision params: 2.239418e+07
YOU ARE EXCLUDING:  ['optimizer']
model
dict_keys(['model', 'optimizer'])
optimizer
dict_keys(['model', 'optimizer'])
_output_dir
global_step
epoch
dict_keys(['params_dict.action.offset', 'params_dict.action.scale', 'params_dict.action.input_stats.max', 'params_dict.action.input_stats.mean', 'params_dict.action.input_stats.min', 'params_dict.action.input_stats.std', 'params_dict.base_pose.offset', 'params_dict.base_pose.scale', 'params_dict.base_pose.input_stats.max', 'params_dict.base_pose.input_stats.mean', 'params_dict.base_pose.input_stats.min', 'params_dict.base_pose.input_stats.std', 'params_dict.arm_pos.offset', 'params_dict.arm_pos.scale', 'params_dict.arm_pos.input_stats.max', 'params_dict.arm_pos.input_stats.mean', 'params_dict.arm_pos.input_stats.min', 'params_dict.arm_pos.input_

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
policy = policy.to(device)
policy.eval()
for param in policy.parameters():
    param.requires_grad = False
pw = PolicyWrapper(policy, n_obs=2, n_acts=8, d_pos=6, d_rot=6, device=device)

In [5]:
obs = {
    'base_pose': np.zeros(3),
    'arm_pos': np.zeros(3),
    'arm_quat': np.zeros(4),
    'gripper_pos': np.zeros(1),
    'base_image': np.zeros((84, 84, 3), dtype=np.uint8),
    'wrist_image': np.zeros((84, 84, 3), dtype=np.uint8),
}

In [9]:
action = pw.get_action(obs)
print(action)

[ 0.15361279  0.00192948  0.00346004  1.         -0.13698481 -0.02300356
  0.09278427  0.915581   -0.3452509   0.04831392  1.          1.
  0.46922943  0.46922943]


In [ ]:
# TODO: generalize this to any environment
base_pose = action[:3]
arm_pos = action[3:6]
arm_6d = torch.from_numpy(action[6:12])
arm_quat = rmat_to_quat(rot6d_to_rmat(arm_6d))
gripper_pos = action[12:13]

action_dict = {
    'base_pose': base_pose,
    'arm_pos': arm_pos,
    'arm_quat': arm_quat,
    'gripper_pos': gripper_pos
}